# driver_DD_NMROM
Driver to implement and test NM ROM on the 2D Burgers Equation.  

In [ ]:
import sys
with open("./../../PATHS.txt") as file:
  paths = file.read().splitlines()
sys.path.extend(paths)

In [ ]:
import os
import numpy as np
import dill as pickle
import scipy.sparse as sp
import scipy.linalg as la
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import dill as pickle

from matplotlib import cm

In [ ]:
from dd_nm_rom.fom import Burgers2D, DDBurgers2D, Burgers2DExact
from dd_nm_rom.rom import DD_NM_ROM
from dd_nm_rom.rom.linear.domain_dec import compute_bases_from_svd

In [ ]:
path_to_run = "/Users/zanardi1/Workspace/Codes/DD-NM-ROM/run/"
data_dir = path_to_run + '/data/'
fig_dir0 = path_to_run + '/figures/'
for d in [data_dir, fig_dir0]:
    os.makedirs(d, exist_ok=True)
plt.rc('font', size=20)
plt.rcParams['text.usetex'] = False

## Set model parameters

In [ ]:
# define constant parameters for PDE
nx, ny  = 480, 24
x_lim   = [-1.0, 1.0]
y_lim   = [0.0, 0.05]

na1, nlam = 80, 80
a1_lim  = [1.0, 10000.0]
lam_lim = [5.0, 25.0]

a1, lam = 7692.5384, 21.9230
viscosity = 1e-1

print(f'a1, lam = {a1, lam}')

# number of subdomains in x and y directions for DD model
n_sub_x = 2
n_sub_y = 2

scaling = -1 # scaling factor for residual. -1 uses hx*hy

n_sub   = n_sub_x*n_sub_y
fig_dir1 = fig_dir0 + f'/nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y/'
fig_dir  = fig_dir1 + f'/a1_{a1}_lam_{lam}/'
svd_dir  = path_to_run + f'/linear/data/nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y/'
for d in [fig_dir1, fig_dir]:
    os.makedirs(d, exist_ok=True)
if not os.path.exists(svd_dir): print('Error: Must run LS-ROM notebook before proceeding.')

In [ ]:
# load snapshot and residual data
file = data_dir + f'/residual/nx{nx}_ny{ny}_mu{viscosity}_nsamples{400}.p'
data = pickle.load(open(file, 'rb'))
Mu = data['parameters']
snapshots = data['snapshots']
residuals = data['residuals']

## Solve DD FOM

In [ ]:
burg_ex = Burgers2DExact()
burg_ex.set_params(a1, lam, viscosity)

In [ ]:
print('\nSolving full domain model:')

# generate Burgers FOM
fom = Burgers2D(nx, ny, x_lim, y_lim, viscosity)
fom.set_bc(burg_ex.u, burg_ex.v)
uv, rhs = fom.solve(tol=1e-8, maxit=20, stepsize_min=1e-20, verbose=True)
sol = np.concatenate([uv["u"], uv["v"]])
print("RUNTIME:", fom.runtime)

print('\nSolving DD model:')
ddmdl = DDBurgers2D(fom, n_sub_x, n_sub_y, scaling=-1)
ddmdl.set_bc()
ndof_fom = ddmdl.get_ndof()

print(f'\nFOM DoF = {ndof_fom-ddmdl.n_constraints}')

uv_dd, lambdas = ddmdl.solve(x0=np.zeros(ndof_fom), tol=1e-8, maxit=100, verbose=True)
sol_dd = np.concatenate([uv_dd["res"]["u"], uv_dd["res"]["v"]])
print("RUNTIME:", ddmdl.runtime)

# dd_fom_rel_err = np.linalg.norm(sol_dd-sol)/np.linalg.norm(sol)
dd_fom_rel_err = 100*np.mean(np.abs(sol_dd-sol)/(np.abs(sol)+1e-7))
print(f'\nDD-FOM mean relative error = {dd_fom_rel_err:1.4e} %') 

In [ ]:
# plot DD FOM u and v
x = np.linspace(x_lim[0], x_lim[1], nx+2)[1:-1]
y = np.linspace(y_lim[0], y_lim[1], ny+2)[1:-1]
X, Y = np.meshgrid(x, y)
U_fom = uv_dd["res"]["u"].reshape(ny, nx)
V_fom = uv_dd["res"]["v"].reshape(ny, nx)

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, U_fom, cmap=cm.jet, shading='auto', vmin=uv["u"].min(), vmax=uv["u"].max())
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$u$')
plt.tight_layout()
file = fig_dir+'u_fom.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, V_fom, cmap=cm.jet, shading='auto', vmin=uv["v"].min(), vmax=uv["v"].max())
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$v$')
plt.tight_layout()
file = fig_dir+'v_fom.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Solve DD NM-ROM

In [ ]:
ec     = 1e-10          # energy criteria for number of interior and interface basis vectors
nbasis = -1             # number of residual basis vectors. set to -1 to use energy criteria

# load SVD data
print('Loading residual SVD data...')
res_dict  = pickle.load(open(svd_dir+'res_svd.p', 'rb'))
print('Data loaded!')

# compute bases
print('Computing residual bases...')
residual_bases = compute_bases_from_svd(res_dict, ec=ec, nbasis=nbasis)
print('Bases computed!')

for i in range(n_sub):
    print(f'residual_bases[{i}]={residual_bases[i].shape}')

In [ ]:
single_model = True # False True
suffix = "_numpy"
name = f'nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y'

# port ROM sizes
pld = 2

# interior and interface state ROM sizes
intr_size = 6
intf_size = 4
intr_rnnz = 5
intf_rnnz = 5
intr_shift = 5
intf_shift = 5
intr_act_type = 'Swish'
intf_act_type = 'Swish'
batch = 32

if single_model:
  pld = 4
  intr_size = 12
  intf_size = 8
  batch = 64

# port states
port_size  = [min(len(ddmdl.port_to_nodes[p]-1), pld) for p in ddmdl.ports]
port_rnnz  = 3*np.ones(len(ddmdl.ports), dtype=int) 
port_shift = 3*np.ones(len(ddmdl.ports), dtype=int) 
port_act_type = 'Sigmoid' #'Swish'

Ntotal = 6400 if nx in [240, 480] else 4200
net_folder0 = data_dir + f'/trained_nets/{name}/multi/ports-iden/'
port_net_list = [net_folder0+\
                 f'port_{i+1}of{len(ddmdl.ports)}/'+\
                 f'ld_{port_size[i]}_rnnz_{port_rnnz[i]}_rshift_{port_shift[i]}_'+\
                 f'{port_act_type}_batch_32_AbsMSEloss_{Ntotal}snaps{suffix}.p' \
                  for i in range(len(ddmdl.ports))]

# interior and interface states
Ntotal = 6400 if nx in [240, 480] else 4200
net_folder0 = data_dir + f'/trained_nets/{name}/multi/'
intr_net_list = [net_folder0+\
                 f'sub_{i+1}of{n_sub}/interior/'+\
                 f'ld_{intr_size}_rnnz_{intr_rnnz}_rshift_{intr_shift}_'+\
                 f'{intr_act_type}_batch_32_AbsMSEloss_{Ntotal}snaps{suffix}.p' \
                  for i in range(n_sub)]
intf_net_list = [net_folder0+\
                 f'sub_{i+1}of{n_sub}/interface/'+\
                 f'ld_{intf_size}_rnnz_{intf_rnnz}_rshift_{intf_shift}_'+\
                 f'{intf_act_type}_batch_32_AbsMSEloss_{Ntotal}snaps{suffix}.p' \
                  for i in range(n_sub)]

if single_model:

  Ntotal = 6400 if nx in [240, 480] else 4200
  net_folder0 = data_dir + f'/trained_nets/{name}/single/ports/'
  port_net_list = [net_folder0+\
                  f'port_dim_{ddmdl.port_to_nodes[i].size}/'+\
                  f'ld_{port_size[i]}_rnnz_{port_rnnz[i]}_rshift_{port_shift[i]}_'+\
                  f'{port_act_type}_batch_{batch}_AbsMSEloss_{Ntotal}snaps{suffix}.p' \
                    for i in range(len(ddmdl.ports))]
  
  
  Ntotal = 6400 if nx in [240, 480] else 4200
  net_folder0 = data_dir + f'/trained_nets/{name}/single/'
  intr_net_list = [net_folder0 + '/interior/'+\
                  f'ld_{intr_size}_rnnz_{intr_rnnz}_rshift_{intr_shift}_'+\
                  f'{intr_act_type}_batch_{batch}_AbsMSEloss_{Ntotal}snaps{suffix}.p' \
                    for _ in range(n_sub)]
  intf_net_list = [net_folder0 + '/interface/'+\
                  f'ld_{intf_size}_rnnz_{intf_rnnz}_rshift_{intf_shift}_'+\
                  f'{intf_act_type}_batch_{batch}_AbsMSEloss_{Ntotal}snaps{suffix}.p' \
                    for _ in range(n_sub)]

In [ ]:
nn_configfiles = {
  "port": port_net_list,
  "interface": intf_net_list,
  "interior": intr_net_list,
}

In [ ]:
# build ROM and RBF model (used for generating initial iterate)
print('Building DD NM-ROM...')
ddnmrom = DD_NM_ROM(
  dd_fom=ddmdl,
  nn_configfiles=nn_configfiles,
  res_bases=residual_bases,
  hr_active=True,
  hr_n_samples=100,
  hr_n_edge_samples_ratio=0.75,
  hr_sample_small_ports=True,
  hr_small_ports_dim=5,
  constraint_type='weak',
  n_constraints_weak=intf_size,
  seed=1,
  scaling=-1
)
print('ROM built!\n')

In [ ]:
# for sub in ddnmrom.subdomains:
#   samples = np.array([], dtype=np.int32)
#   for p in sub.ports:
#     nodes = sub.port_to_nodes[p]
#     if (nodes.size <= sub.hr_small_ports_dim):
#       inter = np.nonzero(np.isin(sub.indices["res"], nodes))[0]
#       inter = np.concatenate([inter, inter + sub.n_nodes["res"]])
#       samples = np.union1d(samples, inter)
#   print(np.isin(samples, sub.hr_ind_uv["res"]))

In [ ]:
print('Computing RBF interpolant ...')
dd_data = ddmdl.map_sol_on_elements(np.vstack(snapshots), map_on_ports=False)
ddnmrom.init_rbf(dd_data, Mu, smoothing=0.0, kernel='linear')
print('Interpolant computed!')

In [ ]:
# solve DD ROM
print('Solving DD NM-ROM...')
uv_dd_rom, z_rom, lambdas, rhs = ddnmrom.solve(mu=np.array([a1, lam]), tol=1e-8, maxit=50, stepsize_min=1e-10, verbose=True)
print('Solution found!\n')
print("RUNTIME:", ddnmrom.runtime)

# compute error
dd_rom_rel_err = ddnmrom.compute_error(uv_dd, uv_dd_rom)
print(f'DD NM-ROM rel. error = {dd_rom_rel_err:1.4e}')

Plot DD ROM u and v

In [ ]:
nm_figs  = fig_dir + '/nmrom/'
nm_figs += '/single/' if single_model else '/multi/'
os.makedirs(nm_figs, exist_ok=True)

In [ ]:
# plot DD ROM u and v
filename = nm_figs
filename += '/srpc' if ddnmrom.constraint_type == 'strong' else '/wfpc'
filename += '_col_hr' if ddnmrom.hr_active else ''

# plot DD FOM u and v
x = np.linspace(x_lim[0], x_lim[1], nx+2)[1:-1]
y = np.linspace(y_lim[0], y_lim[1], ny+2)[1:-1]
X, Y = np.meshgrid(x, y)
U_rom = uv_dd_rom["res"]["u"].reshape(ny, nx)
V_rom = uv_dd_rom["res"]["v"].reshape(ny, nx)

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, U_rom, cmap=cm.jet, shading='auto', vmin=U_fom.min(), vmax=U_fom.max())
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$\hat{u}$')
plt.tight_layout()
file = filename+'_u.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, V_rom, cmap=cm.jet, shading='auto', vmin=V_fom.min(), vmax=V_fom.max())
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$\hat{v}$')
plt.tight_layout()
file = filename+'_v.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# plot errors
u_rel_err = np.abs(U_fom-U_rom) / np.linalg.norm(U_fom)
plt.figure(figsize=(12,4))
plt.pcolormesh(X, Y, u_rel_err, cmap=cm.jet, shading='auto')#, vmin=0.0, vmax=4.88e-4)
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$u$ error', format='%.0e')
plt.tight_layout()
file = filename+'_u_error.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12,4))
v_rel_err = np.abs(V_fom-V_rom)/np.linalg.norm(V_fom)
plt.pcolormesh(X, Y, v_rel_err, cmap=cm.jet, shading='auto')#, vmin=0.0, vmax=4.88e-4)
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$v$ error', format='%.0e')
plt.tight_layout()
file = filename+'_v_error.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# Check constraints
# FOM
cres_fp= np.zeros(ddmdl.n_constraints)
for i, s in enumerate(ddmdl.subdomains):
    zi = w_dd_rom["interface"][i]
    x_intf = ddnmrom.subdomains[i].decode(x=zi, k="interface")[0]
    cres_fp += s.cmat["interface"]@x_intf

# ROM
cres_rp = np.zeros(ddnmrom.n_constraints)
if ddnmrom.constraint_type == 'strong':
    for i, s in enumerate(ddnmrom.subdomains):
        cres_rp += s.cmat@w_dd_rom["interface"][i]
    print(f'ROM-port constraint residual (SRPC) = {np.linalg.norm(cres_rp):1.4e}')
    print(f'FOM-port constraint residual (SRPC) = {np.linalg.norm(cres_fp):1.4e}')
else:
    for i, s in enumerate(ddnmrom.subdomains):
        zi = w_dd_rom["interface"][i]
        x_intf = ddnmrom.subdomains[i].decode(x=zi, k="interface")[0]
        cres_fp += s.cmat@x_intf
    print(f'ROM-port constraint residual (WFPC) = {np.linalg.norm(cres_rp):1.4e}')
    print(f'FOM-port constraint residual (WFPC)= {np.linalg.norm(cres_fp):1.4e}')

In [ ]:
# compute lagrange multiplier approximation
de, jde = ddnmrom.subdomain[0].de_intf(w_dd_rom["interface"][0])
en, jen = ddnmrom.subdomain[0].en_intf(de)
lam_rom_approx = jen.T@lam_opt if ddnmrom.constraint_type=='strong' else ddnmrom.constraint_mult.T@lam_opt

# compute error estimate
xtilde = []
xhat = []
for i, s in enumerate(ddnmrom.subdomain):
    xtilde.append(s.de_intr(w_intr[i])[0])
    xtilde.append(s.de_intf(w_dd_rom["interface"][i])[0])
    xhat.append(w_intr[i])
    xhat.append(w_dd_rom["interface"][i])
    
xtilde.append(lam_rom_approx)
xtilde = np.concatenate(xtilde)
xhat.append(lam_opt)
xhat = np.concatenate(xhat)

f, j, rt = ddmdl.FJac(xtilde)
e = np.linalg.norm(sp.linalg.spsolve(j, f))

print(f'norm of FOM lagrange mult. = {np.linalg.norm(lam_fom):1.4e}')
print(f'norm of ROM lagrange mult. = {np.linalg.norm(lam_rom_approx):1.4e}')
print(f'Lagrange mult. error       = {np.linalg.norm(lam_fom-lam_rom_approx):1.4e}')
print(f"||(F'(g(x))^(-1)F(g(x)||   = {np.linalg.norm(e):1.4e}")

In [ ]:
xstar = []
for i, s in enumerate(ddmdl.subdomain):
    xstar.append(u_intr[i])
    xstar.append(v_intr[i])    
    xstar.append(u_intf[i])
    xstar.append(v_intf[i]) 
xstar.append(lam_fom)
xstar = np.concatenate(xstar)

f, j, rt = ddmdl.FJac(xstar)
np.linalg.norm(sp.linalg.spsolve(j, f))

## Training time, number of parameters per subdomain

In [ ]:
print('subdomain       interior          interface')
for i, s in enumerate(ddnmrom.subdomain):
    print(f'    {i}          {s.train_time_intr:1.4e}         {s.train_time_intr:1.4e}')

In [ ]:
intr_params = []
intf_params = []
print('subdomain     interior     interface')
for i, s in enumerate(ddnmrom.subdomain):
    pcount = s.en_W1_intr.nnz+np.prod(s.en_W2_intr.shape)+s.en_b1_intr.size
    pcount += s.de_W2_intr.nnz+np.prod(s.de_W1_intr.shape)+s.de_b1_intr.size
    intr_params.append(pcount)
    
    pcount = s.en_W1_intf.nnz+np.prod(s.en_W2_intf.shape)+s.en_b1_intf.size
    pcount += s.de_W2_intf.nnz+np.prod(s.de_W1_intf.shape)+s.de_b1_intf.size
    intf_params.append(pcount)
    print(f'    {i}          {intr_params[i]}         {intf_params[i]}')

In [ ]:
maxp_per_subdomain=np.max(np.array(intr_params+intf_params))
all_params=np.sum(np.array(intr_params+intf_params))
all_params

In [ ]:
maxp_per_subdomain

In [ ]:
max_train_time = np.max([s.train_time_intr for s in ddnmrom.subdomain]+\
                        [s.train_time_intf for s in ddnmrom.subdomain])
max_train_time

In [ ]:
intr_params+intf_params

## Spy plots of sparsity masks

In [ ]:
# plot sparsity masks
sub = 0
plt.figure(figsize=(12, 4))
plt.spy(ddnmrom.subdomain[sub].de_mask_intr)
file = fig_dir+f'ld_{ddnmrom.subdomain[sub].intr_latent_dim}_rnnz_{intr_rnnz}_rshift_{intr_shift}_intr_mask.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12, 8))
plt.spy(ddnmrom.subdomain[sub].de_mask_intf)
plt.xticks([])
plt.yticks([])
file = fig_dir+f'ld_{ddnmrom.subdomain[sub].intf_latent_dim}'+ \
               f'_rnnz_{port_rnnz[0]}_rshift_{port_shift[0]}_intf_mask{ct_str}.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Check reconstruction error

In [ ]:
error = 0.0
intr_errors = []
intf_errors = []

for i, s in enumerate(ddnmrom.subdomain):
    uv_intr = np.concatenate([u_intr[i], v_intr[i]])
    uv_intf = np.concatenate([u_intf[i], v_intf[i]])
    
    num_intr = np.sum(np.square(uv_intr - s.de_intr(s.en_intr(uv_intr))[0])) 
    num_intf = np.sum(np.square(uv_intf - s.de_intf(s.en_intf(uv_intf))[0])) 
    den_intr = np.sum(np.square(uv_intr)) 
    den_intf =  np.sum(np.square(uv_intf)) 
    intr_errors.append(num_intr/den_intr)
    intf_errors.append(num_intf/den_intf)
    error += (num_intr+num_intf)/(den_intr+den_intf)
    
error = np.sqrt(error/ddnmrom.n_sub)
print(f'Relative reconstruction error = {error:1.4e}\n')
print('Subdomain     Intr. rel. error     Intf. rel. error')
for i in range(n_sub):
    print(f'    {i+1:2d}           {intr_errors[i]:1.4e}           {intf_errors[i]:1.4e}') 

## Analyze behavior of residual at FOM and ROM solutions

In [ ]:
# compute ROM residual, equality constraint
Ag_rom = np.zeros(ddnmrom.n_constraints)
res_rom = 0.0
res_list = []
for j, s in enumerate(ddnmrom.subdomain):
    res, jac, H, rhs, Ag, Adg = s.res_jac(w_intr[j], w_dd_rom["interface"][j], lam_opt)
    res_list.append(res)
    res_rom += np.dot(res, res)
    Ag_rom += Ag
res_rom *= 0.5

# compute FOM residual, equality constraint
Ag_fom  = np.zeros(ddmdl.n_constraints)
res_fom = 0.0
for j, s in enumerate(ddmdl.subdomain):
    res_fom += np.linalg.norm(s.res_jac(u_intr[j], v_intr[j], u_intf[j], v_intf[j], lam_fom)[0])**2
    Ag_fom += s.cmat@np.concatenate([u_intf[j], v_intf[j]])
res_fom *= 0.5

print(f'ROM LSTSQ residual  = {res_rom:1.4e}')
print(f'ROM constraint norm = {np.linalg.norm(Ag_rom):1.4e}')
print(f'FOM LSTSQ residual  = {res_fom:1.4e}')
print(f'FOM constraint norm = {np.linalg.norm(Ag_fom):1.4e}')

In [ ]:
# compute residual, constraint at encoded FOM solution 
Ag_en = np.zeros(ddnmrom.n_constraints)
res_en = 0.0
en_res_list = []
for j, s in enumerate(ddnmrom.subdomain):
    en_uv_intr = s.en_intr(np.concatenate((u_intr[j], v_intr[j])))
    en_uv_intf = s.en_intf(np.concatenate((u_intf[j], v_intf[j])))
    
    res, jac, H, rhs, Ag, Adg = s.res_jac(en_uv_intr, en_uv_intf, lam_opt)
    en_res_list.append(res)
    res_en += np.dot(res, res)
    Ag_en += Ag
res_en *= 0.5
print(f'ROM LSTSQ residual at encoded FOM sol.  = {res_en:1.4e}')
print(f'ROM constraint norm at encoded FOM sol. = {np.linalg.norm(Ag_en):1.4e}')

In [ ]:
x_err_list = []
res_fom_list = []
res_enc_list = []
RTr_fom_list = []
RTr_enc_list = []
for j, s in enumerate(ddmdl.subdomain):
    res, jac, H, rhs, Ax = s.res_jac(u_intr[j], v_intr[j], u_intf[j], v_intf[j], lam_fom)
    RTr = jac.T@res #np.concatenate([jac_intr.T@res, jac_intf.T@res])
    RTr_fom_list.append(np.linalg.norm(RTr))
    res_fom_list.append(np.linalg.norm(res))
    
    uv_intr_enc = np.concatenate((u_intr[j], v_intr[j]))
    uv_intf_enc = np.concatenate((u_intf[j], v_intf[j]))
    
    uv_intr_enc = ddnmrom.subdomain[j].de_intr(ddnmrom.subdomain[j].en_intr(uv_intr_enc))[0]
    uv_intf_enc = ddnmrom.subdomain[j].de_intf(ddnmrom.subdomain[j].en_intf(uv_intf_enc))[0]
    
    u_intr_enc = uv_intr_enc[:s.n_interior]
    v_intr_enc = uv_intr_enc[s.n_interior:] 
    u_intf_enc = uv_intf_enc[:s.n_interface]
    v_intf_enc = uv_intf_enc[s.n_interface:]
    
    res, jac, H, rhs, Ax = s.res_jac(u_intr_enc, v_intr_enc, u_intf_enc, v_intf_enc, lam_fom)
    RTr = jac.T@res #np.concatenate([jac_intr.T@res, jac_intf.T@res])
    RTr_enc_list.append(np.linalg.norm(RTr))
    res_enc_list.append(np.linalg.norm(res))
    
    x_err_list.append(np.linalg.norm(np.concatenate([u_intr[j], v_intr[j], u_intf[j], v_intf[j]])-
                                    np.concatenate([uv_intr_enc, uv_intf_enc])))
    
print(f"sub.     ||x-x_enc|||    ||r(x)||      ||r(x_enc)||     ||R'(x)^Tr(x)||    ||R'(x_enc)^Tr(x_enc)||")
for j in range(n_sub):
    print(f' {j:d}       {x_err_list[j]:1.3e}       {res_fom_list[j]:1.3e}     {res_enc_list[j]:1.3e}         {RTr_fom_list[j]:1.3e}         {RTr_enc_list[j]:1.3e}')    

In [ ]:
print(f"itr     ||R'*r||         ||Ag||")
for j, r in enumerate(rhs_vecs):
    Rr = np.linalg.norm(r[:-ddnmrom.n_constraints])
    Ag = np.linalg.norm(r[-ddnmrom.n_constraints:])    
    print(f' {j:2d}     {Rr:1.6e}     {Ag:1.6e}')

In [ ]:
# plot residuals on FD grid
nx_sub, ny_sub = nx//n_sub_x, ny//n_sub_y
nxy_sub = nx_sub*ny_sub
if ddnmrom.hr:
    u_res_list = []
    v_res_list = []
    for i, s in enumerate(ddnmrom.subdomain):
        r = np.zeros(2*s.n_residual)+1e-16
        r[s.hr_ind] = res_list[i]
        u_res_list.append(r[:nxy_sub].reshape(ny_sub, nx_sub))
        v_res_list.append(r[nxy_sub:].reshape(ny_sub, nx_sub))
else:
    u_res_list = [r[:nxy_sub].reshape(ny_sub, nx_sub) for r in res_list]
    v_res_list = [r[nxy_sub:].reshape(ny_sub, nx_sub) for r in res_list]

Ures = []
Vres = []
counter = 0
for j in range(n_sub_y):
    Urow = []
    Vrow = []
    for i in range(n_sub_x):
        sub = n_sub_x*j+i
        Urow.append(u_res_list[sub])
        Vrow.append(v_res_list[sub])
    Ures.append(Urow)
    Vres.append(Vrow)
Ures = np.abs(np.bmat(Ures))
Vres = np.abs(np.bmat(Vres))        

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, Ures, norm=colors.LogNorm(vmin=Ures.min(), vmax=Ures.max()), cmap=cm.jet)
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$u$ residual')
plt.show()

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, Vres, norm=colors.LogNorm(vmin=Vres.min(), vmax=Vres.max()), cmap=cm.jet)
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$v$ residual')
plt.show()

In [ ]:
# net_folder1 = './trained_nets/nx_480_ny_24_mu_0.1_2x_by_2y/'
# net_name = 'ld_10_rnnz_10_rshift_10_Sigmoid_batch_32_AbsMSEloss_4200snaps.p'
# filename = net_folder1 + 'sub_1of4/interior/' + net_name
# net_dict = torch.load(filename)
# de_net = net_dict['decoder']
# de_net

In [ ]:
# if 'net.2.bias' in de_net:
#     print('Deleting old NNs...')
#     for phase in ['interior', 'interface']:
#         for sub in range(n_sub):
#             os.remove(net_folder1+f'sub_{sub+1}of{n_sub}/{phase}/'+net_name)
#     print('All done!')
# else:
#     print('all good')

In [ ]:
w_opt = np.array([])
for i in range(ddnmrom.n_sub):
    w_opt = np.concatenate([w_opt, w_intr[i], w_dd_rom["interface"][i]])
w_opt = np.concatenate([w_opt, lam_opt])

eig_list = []
for vec in [np.zeros(w0.shape), w_rbf, w_opt]:
    rhs, KKT, rjtime = ddnmrom.FJac(vec)
    eigs = la.eigh(KKT.toarray(), eigvals_only = True)
    eig_list.append(eigs/eigs.max())
    
plt.figure(figsize=(10, 8))
plt.scatter(np.arange(eig_list[0].size), eig_list[0], s=100, c='blue', marker='o', label='zeros', alpha=0.75)
plt.scatter(np.arange(eig_list[0].size), eig_list[1], s=100, c='red', marker='^', label='RBF initial', alpha=0.75)
plt.scatter(np.arange(eig_list[0].size), eig_list[2], s=100, c='green', marker='x', label='Final', alpha=0.75)
plt.yscale('log')
# plt.xscale('log')
plt.ylabel('Normalized Eigenvalues')
plt.xlabel('$k$')
plt.grid()
plt.legend()
plt.show()

print(f'# DoF = {ndof_rom}')
print(f'# constraints = {ddnmrom.n_constraints}\n')
print('          Zeros     RBF      Final')
print(f'# Pos.      {np.sum(eig_list[0]>0):2d}       {np.sum(eig_list[1]>0):2d}        {np.sum(eig_list[2]>0):2d}')
print(f'# Neg.      {np.sum(eig_list[0]<0):2d}       {np.sum(eig_list[1]<0):2d}        {np.sum(eig_list[2]<0):2d}')